<a href="https://colab.research.google.com/github/MiladQolami/Neural-Data-Analysis/blob/main/SpikingVariablity/SpikingVariablity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⭕ spiking variablity

---











### Dataset:
The data for this notebook comes from [Allen Institute's Visual Coding – Neuropixels project](https://allensdk.readthedocs.io/en/latest/visual_coding_neuropixels.html) which is an open dataset. In this project spikes were recorded from several regions of mouse brian. The dataset is huge so we just upload small proportion of it: spike form mouse V1 in response to full-field flashes of light/dark.

In [ ]:
# @markdown Install packages and import libraries
!pip install allensdk       # install allensdk, a module for
                            # accessing and processing Allen institute datasets

# Import pachages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 

# Import EcephysProjectCache from allensdk
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache



## Irigularity of inter-spike-interval (ISI)


## Trial-to-tiral variablity





## Spike count corelation